In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests

from io import StringIO

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2155
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2155
done

# All requested packages already installed.

Libraries imported.


## Data

data came from https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/?refine.state=TX since no cross reference or webpage with the needed info existed. created own data set hosted here: https://drive.google.com/file/d/10lzwlNkK2f8hNccJlU95tR4Ta_zMVJRd/view?usp=sharing

In [3]:
#import csv from google drive
url = 'https://drive.google.com/file/d/10lzwlNkK2f8hNccJlU95tR4Ta_zMVJRd/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]


austin_raw = pd.read_csv(path)
austin_raw.head()

,Zip,City,Latitude,Longitude
0,78789,Austin,30.326374,-97.771258
1,78773,Austin,30.326374,-97.771258
2,78709,Austin,30.326374,-97.771258
3,78745,Austin,30.207559,-97.795750
4,78759,Austin,30.406169,-97.757430


In [4]:
print('The data frame has {} zip codes for the city of Austin'.format(len(austin_raw['Zip'].unique()),
                                                                      austin_raw.shape[0]))

The data frame has 83 zip codes for the city of Austin


In [5]:
#define geocoder

address = 'Austin, Tx'

geolocator = Nominatim(user_agent="aus_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Austin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Austin are 30.2711286, -97.7436995.


In [6]:
#create map of Austin using Lat and Long Value
aus_map = folium.Map(location=[latitude, longitude], zoom_start=10)

In [7]:
# add markers to map
for Latitude, Longitude, Zip in zip(austin_raw['Latitude'], austin_raw['Longitude'], austin_raw['Zip']):
    label = '{}, {}'.format(austin_raw, zip)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(aus_map)  
    
aus_map

In [8]:
#defind foursquare credentials
CLIENT_ID = 'VXLNHTMDSQBAGQWIKCQ2T2OCR3MYLHJXFN55SR5VVJJNPL1V' # your Foursquare ID
CLIENT_SECRET = 'RWRDDNPGQ3BS015I50EUUTUYMMKVJU0MGVLN2JIBQQZUAURT' # your Foursquare Secret
VERSION = '20210422' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [9]:
austin_raw.loc[0, 'Zip']

78789

In [10]:
#pulling zip codes lat and long
neighborhood_latitude = austin_raw.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = austin_raw.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_zip = austin_raw.loc[0, 'Zip'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_zip, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of 78789 are 30.326374, -97.771258.


In [11]:
#pull 100 venues that are close to 78789

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VXLNHTMDSQBAGQWIKCQ2T2OCR3MYLHJXFN55SR5VVJJNPL1V&client_secret=RWRDDNPGQ3BS015I50EUUTUYMMKVJU0MGVLN2JIBQQZUAURT&v=20210422&ll=30.326374,-97.771258&radius=500&limit=100'

In [12]:
#show results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60833e6cc6bf3339c2b6828b'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Austin',
  'headerFullLocation': 'Austin',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 30.330874004500007,
    'lng': -97.7660543462985},
   'sw': {'lat': 30.321873995499995, 'lng': -97.7764616537015}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '579fff12498e5310b365cdd1',
       'name': 'Ridge Oak Reservoir Park',
       'location': {'lat': 30.33014,
        'lng': -97.77024,
        'labeledLatLngs': [{'label': 'display',
          'lat': 30.33014,
          'lng': -97.77024}],
 

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Ridge Oak Reservoir Park,Park,30.330140,-97.770240
1,Oates Oasis,Lake,30.322485,-97.773685


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## Function that loops through all zip codes

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=8000): #radius is 5 miles from lat long of zip code
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip', 
                  'Zip Latitude', 
                  'Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
austin_venues = getNearbyVenues(names=austin_raw['Zip'],
                                   latitudes=austin_raw['Latitude'],
                                   longitudes=austin_raw['Longitude']
                                  )

78789
78773
78709
78745
78759
78774
78703
78769
78747
78756
78736
78765
78701
78705
78766
78719
78711
78726
78729
78748
73301
78741
78767
78720
78704
78738
78749
78785
78735
78727
78762
78763
78715
78731
78714
78722
78721
78787
78716
78768
78734
78739
78725
78772
78733
78723
78708
78744
78783
73344
78760
78782
78718
78780
78717
78771
78781
78753
78779
78761
78746
78972
78730
78702
78742
78786
78737
78728
78732
78754
78712
78751
78752
78724
78764
78758
78757
78778
78755
78713
78788
78710
78750


In [18]:
print(austin_venues.shape)
austin_venues.head()

(8039, 7)


,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,78789,30.326374,-97.771258,Covert Park at Mt. Bonnell,30.321119,-97.773482,Park
1,78789,30.326374,-97.771258,Mount Bonnell,30.320932,-97.773316,Mountain
2,78789,30.326374,-97.771258,Mayfield Park and Nature Preserve,30.312824,-97.771504,Park
3,78789,30.326374,-97.771258,Shoal Creek Nursery,30.327314,-97.748036,Flower Shop
4,78789,30.326374,-97.771258,Fonda San Miguel,30.325255,-97.743527,Mexican Restaurant


In [19]:
#group by zip code
austin_venues.groupby('Zip').count()

,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip,,,,,,
73301,100,100,100,100,100,100
73344,100,100,100,100,100,100
78701,100,100,100,100,100,100
78702,100,100,100,100,100,100
78703,100,100,100,100,100,100
78704,100,100,100,100,100,100
78705,100,100,100,100,100,100
78708,100,100,100,100,100,100
78709,100,100,100,100,100,100


In [20]:
#group by category
austin_venues.groupby('Venue Category').count()

,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Adult Boutique,3,3,3,3,3,3
African Restaurant,1,1,1,1,1,1
Airport,4,4,4,4,4,4
Airport Lounge,7,7,7,7,7,7
Airport Service,2,2,2,2,2,2
American Restaurant,246,246,246,246,246,246
Arcade,7,7,7,7,7,7
Argentinian Restaurant,4,4,4,4,4,4
Art Gallery,4,4,4,4,4,4


In [21]:
#find unique categories
print('There are {} uniques categories.'.format(len(austin_venues['Venue Category'].unique())))

There are 247 uniques categories.


## Analyze Zip Codes


In [22]:
# one hot encoding
austin_onehot = pd.get_dummies(austin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
austin_onehot['Zip'] = austin_venues['Zip'] 

# move neighborhood column to the first column
fixed_columns = [austin_onehot.columns[-1]] + list(austin_onehot.columns[:-1])
austin_onehot = austin_onehot[fixed_columns]

austin_onehot.head()

,Zip,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Shop,Bike Trail,Boat Launch,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Cheese Shop,Chinese Restaurant,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Other Great Outdoors,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Picnic Area,Pizza Place,Playground,Pool,Print Shop,Pub,RV Park,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Road,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,78789,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,78789,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [23]:
austin_onehot.shape

(8039, 248)

In [24]:
#Next, let's group rows by zip and by taking the mean of the frequency of occurrence of each category
austin_grouped = austin_onehot.groupby('Zip').mean().reset_index()
austin_grouped

,Zip,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Shop,Bike Trail,Boat Launch,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Cheese Shop,Chinese Restaurant,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Other Great Outdoors,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Picnic Area,Pizza Place,Playground,Pool,Print Shop,Pub,RV Park,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,Road,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,73301,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.01,0.000000,0.010000,0.00,0.00,0.030000,0.000000,0.030000,0.00,0.030000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.040000,0.01,0.000000,0.020000,0.00,0.01,0.000000,0.00,0.00,0.00,0.01,0.000000,0.00,0.00,0.00,0.01,0.020000,0.00,0.02,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.

In [25]:
#confirming df new size
austin_grouped.shape

(83, 248)

In [34]:
#showing each zip code with the 5 most common places in each

num_top_venues = 5

for hood in austin_grouped['Zip']:
    print(hood)
    temp = austin_grouped[austin_grouped['Zip'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


73301
                 venue  freq
0          Pizza Place  0.06
1           Taco Place  0.05
2       Ice Cream Shop  0.05
3                 Park  0.05
4  American Restaurant  0.04


73344
                 venue  freq
0          Pizza Place  0.06
1           Taco Place  0.05
2       Ice Cream Shop  0.05
3                 Park  0.05
4  American Restaurant  0.04


78701
         venue  freq
0        Hotel  0.07
1  Coffee Shop  0.05
2  Yoga Studio  0.04
3   Restaurant  0.04
4   Food Truck  0.04


78702
                venue  freq
0         Coffee Shop  0.08
1               Hotel  0.07
2          Food Truck  0.05
3  Italian Restaurant  0.04
4          Restaurant  0.04


78703
            venue  freq
0            Park  0.06
1           Hotel  0.06
2     Coffee Shop  0.05
3  Ice Cream Shop  0.03
4   Grocery Store  0.03


78704
            venue  freq
0     Coffee Shop  0.07
1           Hotel  0.06
2      Food Truck  0.05
3  Ice Cream Shop  0.04
4      Restaurant  0.04


78705
            venu

            venue  freq
0     Pizza Place  0.06
1  Ice Cream Shop  0.05
2      Taco Place  0.05
3             Bar  0.05
4       Juice Bar  0.04


78757
                venue  freq
0         Pizza Place  0.06
1  Mexican Restaurant  0.06
2             Brewery  0.06
3        Burger Joint  0.06
4          Taco Place  0.05


78758
                   venue  freq
0                Brewery  0.05
1     Mexican Restaurant  0.04
2         Sandwich Place  0.04
3           Burger Joint  0.04
4  Vietnamese Restaurant  0.04


78759
                     venue  freq
0            Grocery Store  0.04
1  New American Restaurant  0.04
2      American Restaurant  0.04
3   Furniture / Home Store  0.04
4         Asian Restaurant  0.04


78760
                 venue  freq
0          Pizza Place  0.06
1           Taco Place  0.05
2       Ice Cream Shop  0.05
3                 Park  0.05
4  American Restaurant  0.04


78761
                 venue  freq
0          Pizza Place  0.06
1           Taco Place  0.05
2  

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
#show top 10 venues in each zip code

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Zip'] = austin_grouped['Zip']

for ind in np.arange(austin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(austin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,73301,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
1,73344,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
2,78701,Hotel,Coffee Shop,Yoga Studio,Restaurant,Food Truck,Café,Bar,American Restaurant,Park,Cocktail Bar
3,78702,Coffee Shop,Hotel,Food Truck,Italian Restaurant,Restaurant,Bar,Cocktail Bar,BBQ Joint,Brewery,Beer Garden
4,78703,Park,Hotel,Coffee Shop,Ice Cream Shop,Grocery Store,Seafood Restaurant,Bar,Movie Theater,Yoga Studio,American Restaurant


## Cluster Zip Codes

In [71]:
# initial number of clusters
kclusters = 5

austin_grouped_clustering = austin_grouped

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(austin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [72]:
#new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

austin_merged = austin_raw

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
austin_merged = austin_merged.join(neighborhoods_venues_sorted.set_index('Zip'), on='Zip')

austin_merged.head() # check the last columns!

,Zip,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78789,Austin,30.326374,-97.771258,2,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
1,78773,Austin,30.326374,-97.771258,2,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
2,78709,Austin,30.326374,-97.771258,0,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
3,78745,Austin,30.207559,-97.795750,4,Taco Place,Coffee Shop,Food Truck,Sandwich Place,Pizza Place,Burger Joint,Fast Food Restaurant,Ice Cream Shop,Bar,Grocery Store
4,78759,Austin,30.406169,-97.757430,4,Grocery Store,New American Restaurant,American Restaurant,Furniture / Home Store,Asian Restaurant,Mexican Restaurant,Taco Place,Cosmetics Shop,Burger Joint,Sporting Goods Shop


In [73]:
austin_merged

,Zip,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78789,Austin,30.326374,-97.771258,2,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
1,78773,Austin,30.326374,-97.771258,2,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
2,78709,Austin,30.326374,-97.771258,0,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
3,78745,Austin,30.207559,-97.795750,4,Taco Place,Coffee Shop,Food Truck,Sandwich Place,Pizza Place,Burger Joint,Fast Food Restaurant,Ice Cream Shop,Bar,Grocery Store
4,78759,Austin,30.406169,-97.757430,4,Grocery Store,New American Restaurant,American Restaurant,Furniture / Home Store,Asian Restaurant,Mexican Restaurant,Taco Place,Cosmetics Shop,Burger Joint,Sporting Goods Shop
5,78774,Austin,30.326374,-97.771258,2,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
6,78703,Austin,30.290907,-97.762770,0,Park,Hotel,Coffee Shop,Ice Cream Shop,Grocery Store,Seafood Restaurant,Bar,Movie Theater,Yoga Studio,American Restaurant
7,78769,Austin,30.326374,-97.771258,2,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
8,78747,Austin,30.132855,-97.761870,4,Mexican Restaurant,Pizza Place,Bar,Food Truck,Chinese Restaurant,Spa,BBQ Joint,Cosmetics Shop,Convenience Store,Sandwich Place
9,78756,Austin,30.320206,-97.741770,4,Pizza Place,Ice Cream Shop,Taco Place,Bar,Juice Bar,Park,Grocery Store,Coffee Shop,Bakery,Sandwich Place


In [74]:
# create map and visulize resulting clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(austin_merged['Latitude'], austin_merged['Longitude'], austin_merged['Zip'],austin_merged['Cluster Labels'] ):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [75]:
#cluster 1
austin_merged.loc[austin_merged['Cluster Labels'] == 0, austin_merged.columns[[0] + list(range(5, austin_merged.shape[1]))]]

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,78709,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
6,78703,Park,Hotel,Coffee Shop,Ice Cream Shop,Grocery Store,Seafood Restaurant,Bar,Movie Theater,Yoga Studio,American Restaurant
12,78701,Hotel,Coffee Shop,Yoga Studio,Restaurant,Food Truck,Café,Bar,American Restaurant,Park,Cocktail Bar
13,78705,Hotel,Coffee Shop,Yoga Studio,Pizza Place,Ice Cream Shop,Bar,Seafood Restaurant,Movie Theater,Grocery Store,Taco Place
15,78719,Hotel,Convenience Store,Racetrack,Rental Car Location,Mexican Restaurant,Sandwich Place,Discount Store,Brewery,Coffee Shop,Taco Place
16,78711,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
17,78726,Sushi Restaurant,Pizza Place,Grocery Store,Sandwich Place,BBQ Joint,Burger Joint,Park,Coffee Shop,Liquor Store,Movie Theater
18,78729,Pizza Place,Park,Mexican Restaurant,Grocery Store,Donut Shop,American Restaurant,Ice Cream Shop,Burger Joint,Café,Asian Restaurant
23,78720,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
24,78704,Coffee Shop,Hotel,Food Truck,Ice Cream Shop,Restaurant,Park,Pizza Place,Taco Place,Beer Garden,Seafood Restaurant


In [76]:
#cluster 2
austin_merged.loc[austin_merged['Cluster Labels'] == 1, austin_merged.columns[[0] + list(range(5, austin_merged.shape[1]))]]

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,73301,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
49,73344,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery


In [77]:
#cluster 3
austin_merged.loc[austin_merged['Cluster Labels'] == 2, austin_merged.columns[[0] + list(range(5, austin_merged.shape[1]))]]

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78789,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
1,78773,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
5,78774,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
7,78769,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
11,78765,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
14,78766,Grocery Store,Coffee Shop,Fast Food Restaurant,Mexican Restaurant,Bakery,Gym / Fitness Center,Sandwich Place,Park,Fried Chicken Joint,Seafood Restaurant
22,78767,American Restaurant,Coffee Shop,Taco Place,Mexican Restaurant,Sandwich Place,Trail,Grocery Store,Park,BBQ Joint,Gym
27,78785,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
30,78762,Pizza Place,Taco Place,Ice Cream Shop,Park,American Restaurant,Burger Joint,BBQ Joint,Japanese Restaurant,Juice Bar,Bakery
31,78763,Convenience Store,Fast Food Restaurant,Golf Course,Sandwich Place,General Entertainment,Park,Mexican Restaurant,Gun Range,Lake,Print Shop


In [78]:
#cluster 4
austin_merged.loc[austin_merged['Cluster Labels'] == 3, austin_merged.columns[[0] + list(range(5, austin_merged.shape[1]))]]

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,78972,Coffee Shop,Taco Place,Bakery,Burger Joint,American Restaurant,Brewery,Breakfast Spot,Liquor Store,Restaurant,Mexican Restaurant


In [79]:
#cluster 5
austin_merged.loc[austin_merged['Cluster Labels'] == 4, austin_merged.columns[[0] + list(range(5, austin_merged.shape[1]))]]

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,78745,Taco Place,Coffee Shop,Food Truck,Sandwich Place,Pizza Place,Burger Joint,Fast Food Restaurant,Ice Cream Shop,Bar,Grocery Store
4,78759,Grocery Store,New American Restaurant,American Restaurant,Furniture / Home Store,Asian Restaurant,Mexican Restaurant,Taco Place,Cosmetics Shop,Burger Joint,Sporting Goods Shop
8,78747,Mexican Restaurant,Pizza Place,Bar,Food Truck,Chinese Restaurant,Spa,BBQ Joint,Cosmetics Shop,Convenience Store,Sandwich Place
9,78756,Pizza Place,Ice Cream Shop,Taco Place,Bar,Juice Bar,Park,Grocery Store,Coffee Shop,Bakery,Sandwich Place
10,78736,BBQ Joint,Liquor Store,American Restaurant,Grocery Store,Pizza Place,Coffee Shop,Cosmetics Shop,Mexican Restaurant,Ice Cream Shop,Furniture / Home Store
19,78748,Mexican Restaurant,Park,Food Truck,Bar,Grocery Store,Donut Shop,Pizza Place,Cosmetics Shop,Gym,Sandwich Place
21,78741,Coffee Shop,Food Truck,Hotel,Pizza Place,Restaurant,Ice Cream Shop,Vietnamese Restaurant,BBQ Joint,Cocktail Bar,Café
25,78738,Mexican Restaurant,American Restaurant,Burger Joint,Golf Course,Coffee Shop,Sandwich Place,Park,Furniture / Home Store,Grocery Store,Restaurant
26,78749,Sandwich Place,Coffee Shop,Taco Place,Park,American Restaurant,Grocery Store,Ice Cream Shop,Trail,Mexican Restaurant,Pizza Place
28,78735,Sandwich Place,Taco Place,Coffee Shop,Trail,Mexican Restaurant,Grocery Store,Burger Joint,American Restaurant,Pizza Place,Café
